Seleziono la cartella per le traccie ridotte tramite widget

In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Set the path to your folder
folder_path = "C:/Users/fturi/Desktop/Dati/Test/tools-isocrone-out"

# Get a list of all the subfolders starting with "RID"
subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and f.startswith("RID")]

# Create a dropdown widget to choose the subfolder
subfolder_dropdown = widgets.Dropdown(options=subfolders, description='Subfolder:')
confirm_button = widgets.Button(description='Confirm')

# Create a variable to store the selected subfolder path
selected_subfolder_path = ''

# Function to handle selection of subfolder
def select_subfolder(change):
    global selected_subfolder_path
    selected_subfolder_path = os.path.join(folder_path, change.new)

def confirm_selection(button):
    if selected_subfolder_path:
        display(selected_subfolder_path)
    else:
        print("Please select a subfolder before confirming.")

subfolder_dropdown.observe(select_subfolder, 'value')
confirm_button.on_click(confirm_selection)

# Display the dropdown widget and confirm button
display(subfolder_dropdown, confirm_button)
selected_subfolder_path=selected_subfolder_path.replace("\\","/")

Dropdown(description='Subfolder:', options=('RID_Z0.00135_He0.2499_ML1.90_55555555_AS09a0.DAT', 'RID_Z0.00145_…

Button(description='Confirm', style=ButtonStyle())

'C:/Users/fturi/Desktop/Dati/Test/tools-isocrone-out\\RID_Z0.00145_He0.2501_ML1.90_55555555_AS09a0.DAT'

Carico i dati su un dizionario contenente i vari dataframe(le key di accesso sono nominate come es"Mass_0.76"). va rilanciato dopo la selezione delle cartelle

In [2]:


def read_data_tracce_ridotte(file_path):
    """
    Reads data from a file with reduce tarces and returns a dataframe with specified ['MOD', 'Time', 'LOG_L/Lo', 'LOG_TE_(K)', 'M', '[Fe/H]', 'R', 'logg', 'Dni', 'nimax'] .

    Args:
        file_path (str): The path to the file containing the data.

    Returns:
        pandas.DataFrame: A dataframe containing the data with assigned variable/column names.
    """
    # Define the variable/column names
    variable_names = ['MOD', 'Time', 'LOG_L/Lo', 'LOG_TE_(K)', 'M', '[Fe/H]', 'R', 'logg', 'Dni', 'nimax']
    
    # Read the data from the file into a dataframe
    df = pd.read_csv(file_path, comment='#', delimiter='\s+', header=None, engine='python')
    
    # Assign the variable/column names to the dataframe columns
    df.columns = variable_names
    
    # Return the dataframe
    return df


def extract_metallicity(file_path):
    """
    Reads from the file path the value of metallicity (Z), the value of He, the mixing length (ML), and the code.

    Args:
        file_path (str): The path to the file containing the data.

    Returns:
        An array [z_value, he_value, ml_value, code_value] with the extracted values.
    """
    # Split the file path by directory separators to extract the file name
    file_name = file_path.split("/")[-1]

    # Remove the ".DAT" extension from the file name
    file_name = file_name.replace(".DAT", "")

    # Split the file name by "_" to extract the individual values
    values = file_name.split("_")

    # Extract the metallicity (Z), He, mixing length (ML), and code values
    z_value = values[1][1:]
    he_value = values[2][2:]
    ml_value = values[3][2:]
    code_value = values[-1]

    return [z_value, he_value, ml_value, code_value]

#Adjust the format of the subfolder_path
subfolder_path=selected_subfolder_path


# Create a dictionary to store the dataframes
dataframes = {}

for file_name in os.listdir(subfolder_path):
    
    if file_name.startswith("AOUT_M"):
        
        #Select the file path for each file.dat in the folder
        file_path = os.path.join(subfolder_path, file_name)
        file_path=file_path.replace("\\","/")
    
        # Load the data into a dataframe
        df =read_data_tracce_ridotte(file_path)
        
       # Extract the number from the filename
        file_number = float(file_name.split("_")[1][1:])  # Extract the number after "M" and convert to float
        
        # Format the dataframe name as "Mass_xxx"
        dataframe_name = f"Mass_{file_number:.2f}"
        
        # Store the dataframe in the dictionary with the formatted name
        dataframes[dataframe_name] = df

#Delate the df of support
del df

#Print what i have in the dictionary
print("Le tracce ridotte usate vengono dal file:",subfolder_path)
dataframe_names = list(dataframes.keys())
value_of_start=extract_metallicity(subfolder_path)
print("I valori iniziali da cui parte l'evoluzione per il seguente dataset, sono: Metallicità(",value_of_start[0],"),He(",value_of_start[1],"),Mixing lenght(",value_of_start[2],")")
print("Le masse a disposizione sono:",dataframe_names)
print("le variabili sono:", list(dataframes[dataframe_names[0]].columns))    

Le tracce ridotte usate vengono dal file: C:/Users/fturi/Desktop/Dati/Test/tools-isocrone-out\RID_Z0.00145_He0.2501_ML1.90_55555555_AS09a0.DAT
I valori iniziali da cui parte l'evoluzione per il seguente dataset, sono: Metallicità( 0.00145 ),He( 0.2501 ),Mixing lenght( 1.90 )
Le masse a disposizione sono: ['Mass_0.76', 'Mass_0.77', 'Mass_0.78', 'Mass_0.79', 'Mass_0.80', 'Mass_0.81', 'Mass_0.82', 'Mass_0.83', 'Mass_0.84', 'Mass_0.85', 'Mass_0.86', 'Mass_0.87', 'Mass_0.88', 'Mass_0.89', 'Mass_0.90', 'Mass_0.91', 'Mass_0.92', 'Mass_0.93', 'Mass_0.94', 'Mass_0.95', 'Mass_0.96', 'Mass_0.97', 'Mass_0.98', 'Mass_0.99', 'Mass_1.00', 'Mass_1.02', 'Mass_1.04', 'Mass_1.06', 'Mass_1.08', 'Mass_1.10']
le variabili sono: ['MOD', 'Time', 'LOG_L/Lo', 'LOG_TE_(K)', 'M', '[Fe/H]', 'R', 'logg', 'Dni', 'nimax']


Ora ho a disposizione il dizionario dataframes, dal quale è possibile richiamare i singoli dataframe per singola massa, ad esempio per richiamare la massa 0.76 si fa df_0.76=dataframes["Mass_0.76"], se si vuole ad esempio l'array della luminosità si fa lum=dataframes["Mass_0.76"]["LOG_L/Lo"].